# 03 SDK | 07 Chat with Json

## Use case

Your data is represented by medium to large JSON objects, you want to enable your customers to interact with your data in a conversational manner. You want to enable your customers to ask questions about your data and get responses in a conversational manner. There are few options to implement this:

- **Incorporating a Full-Text Search Engine**:
   - **Elasticsearch or Azure Cognitive Search**: Index the JSON file content using Elasticsearch or Azure Cognitive Search.
   - **ASP.NET Core Web API**: Build a web API to accept natural language queries, process them, and query the search engine for relevant results.

- **Building a Custom Solution with Language Models**:
   - **OpenAI API or Azure OpenAI Service**: Integrate a pre-trained language model to understand user queries
   - **ASP.NET Core Web API**: Create an API to handle user inputs, query the JSON data, and return responses.

This notebook, focuses on the second option, while we will not create a full-fledged solution here, the outline of the solution is provided.

## Solution Approach

In order to be extract dynamic content from `json` objects, we would first need to have the schema and the user query. A prompt to LLM with the schema and the user query would return the response. The response would be parsed to extract the relevant information.

**Potential Prompt**:

```text
You are given a schema of a JSON dataset and a natural language query. Your task is to convert the natural language query into a structured JSON format that specifies the operation, path, field, and filters needed to execute the query on the JSON dataset.

Schema:
{
  "address": {
    "city": "string",
    "country": "string",
    "postalCode": "string",
    "state": "string",
    "street": "string"
  },
  "contactDetails": {
    "phone": "string"
  },
  "currency": "string",
  "customerId": "string",
  "financials": {
    "12MonthsSales": "number",
    "add": "number",
    "cei": "number",
    "creditMemos": "number",
    "dso": "number",
    "dueInvoices": "number",
    "finScore": "number",
    "monthlyFinancials": [
      {
        "add": "number",
        "cei": "number",
        "dso": "number",
        "invoices": "number",
        "month": "number",
        "openAR": "number",
        "payments": "number",
        "year": "number"
      }
    ],
    "overDueInvoices": "number",
    "totalSales": "number"
  },
  "firstInvoiceDate": "string",
  "foreignId": "string",
  "name": "string",
  "sectorNaicsDescription": "string",
  "sectorNaicsId": "string",
  "taxId": "string",
  "verifiedName": "string"
}

Convert the following natural language query into a structured JSON format:

Example Query: "I need the total for all invoices in year 2021."
Example Output:
{
  "operation": "sum",
  "path": "financials.monthlyFinancials",
  "field": "invoices",
  "filters": [
    {
      "field": "year",
      "operator": "==",
      "value": 2021
    }
  ]
}

Query: "Calculate the average sales for the year 2020."
Output:
{
  "operation": "average",
  "path": "financials.monthlyFinancials",
  "field": "invoices",
  "filters": [
    {
      "field": "year",
      "operator": "==",
      "value": 2020
    }
  ]
}

Query: "{User Query}"
Output:
```

## Prerequisites

In [ ]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: System.Text.Json"
#r "nuget: System.Linq"

### Instantiate OpenAI API

In [ ]:
using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);


string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "..", "assets");

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_CHATCOMPLETION_DEPLOYMENTNAME") ?? "SKIT_CHATCOMPLETION_DEPLOYMENTNAME not found";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);

Console.WriteLine($"OpenAI Client created...");

### Json Schema, Object, and Query

In [ ]:
string userQuery = "looking for the total invoices amount for the year 2021";
string jsonSchema = File.ReadAllText(Path.Combine(assetsFolder,"docs", "03_SDK", "sampleSchema.json"));

string jsonObject = File.ReadAllText(Path.Combine(assetsFolder,"docs", "03_SDK", "sampleObject.json"));

### Prompt to extract intent and entities

In [ ]:
var prompt = $@"
You are given a schema of a JSON dataset and a natural language query. Your task is to convert the natural language query into a structured JSON format that specifies the operation, path, field, and filters needed to execute the query on the JSON dataset.

Schema:
{jsonSchema}

Convert the following natural language query into a structured JSON format:

Example Query: ""I need the total for all invoices in year 2021.""
Example Output:
{{
  ""operation"": ""sum"",
  ""path"": ""financials.monthlyFinancials"",
  ""field"": ""invoices"",
  ""filters"": [
    {{
      ""field"": ""year"",
      ""operator"": ""=="",
      ""value"": 2021
    }}
  ]
}}

Example Query: ""Calculate the average sales for the year 2020.""
Example Output:
{{
  ""operation"": ""average"",
  ""path"": ""financials.monthlyFinancials"",
  ""field"": ""invoices"",
  ""filters"": [
    {{
      ""field"": ""year"",
      ""operator"": ""=="",
      ""value"": 2020
    }}
  ]
}}

";

### Method to extract intent using a call to OpenAI API

In [ ]:
public async Task<string> GetStructuredQueryAsync(string userQuery)
{
    string sys_prompt = prompt;
    ChatCompletionsOptions simpleOption = new ChatCompletionsOptions()
    {
    //Request Properties
    ResponseFormat = ChatCompletionsResponseFormat.JsonObject,
    MaxTokens = 500,
    Temperature = 0.7f,
    NucleusSamplingFactor = 0.0f,
    FrequencyPenalty = 0.0f,
    PresencePenalty = 0.0f,
    DeploymentName = chatCompletionDeploymentName
    };

    simpleOption.Messages.Add(new ChatRequestSystemMessage(sys_prompt));
    var userMessage = $"Query: {userQuery}. Output:";
    simpleOption.Messages.Add(new ChatRequestUserMessage( userMessage));

    Response<ChatCompletions> simpleResponse = await openAIClient.GetChatCompletionsAsync(simpleOption);

    // Get the first choice from the response
    ChatCompletions simpleCompletions = simpleResponse.Value;

    return simpleCompletions.Choices[0].Message.Content;

}

### Test intent extraction

In [ ]:

var structuredQuery = await GetStructuredQueryAsync(userQuery);

Console.WriteLine(structuredQuery);

### Json data extraction

This code uses the structured query, or `json action` to perform filtering and aggregation on the JSON data. The `json action` is a structured JSON object that specifies the operation, path, field, and filters needed to execute the query on the JSON dataset.
This supports the following operations:
- `sum`: Calculate the sum of the field.
- `average`: Calculate the average of the field.
- `count`: Count the number of records.

As a side note, simple operation such as obtain specific field value based on a filter could use just json path and filter.

In [ ]:
using System;
using System.Linq;
using System.Text.Json;
using System.Text.Json.Nodes;
using System.Collections.Generic;

// Define a method to parse the JSON object and perform the operation specified in the structured query
public static string ExecuteStructuredQuery(string jsonObject, string structuredQuery)
{
    var jsonData = JsonNode.Parse(jsonObject);
    var query = JsonNode.Parse(structuredQuery);

    // Function to get a JsonArray from a dynamic path
    JsonArray GetJsonArrayFromPath(JsonNode node, string path)
    {
        var parts = path.Split('.');
        foreach (var part in parts)
        {
            node = node[part];
        }
        return node.AsArray();
    }

    // Extract the relevant data using the specified path
    var jsonPath = query["path"].GetValue<string>();
    var data = GetJsonArrayFromPath(jsonData, jsonPath);

    // Function to apply filters
    IEnumerable<JsonNode> ApplyFilters(IEnumerable<JsonNode> data, JsonArray filters)
    {
        foreach (var filter in filters)
        {
            var field = filter["field"].GetValue<string>();
            var operator_ = filter["operator"].GetValue<string>();
            var value = filter["value"];

            data = data.Where(item => 
            {
                var itemValue = item[field];
                return operator_ switch
                {
                    "==" => itemValue.GetValue<int>() == value.GetValue<int>(),
                    "!=" => itemValue.GetValue<int>() != value.GetValue<int>(),
                    ">" => itemValue.GetValue<decimal>() > value.GetValue<decimal>(),
                    "<" => itemValue.GetValue<decimal>() < value.GetValue<decimal>(),
                    ">=" => itemValue.GetValue<decimal>() >= value.GetValue<decimal>(),
                    "<=" => itemValue.GetValue<decimal>() <= value.GetValue<decimal>(),
                    _ => throw new NotSupportedException($"Operator {operator_} is not supported")
                };
            });
        }
        return data;
    }

    // Apply filters from the query
    var filters = query["filters"].AsArray();
    var filteredData = ApplyFilters(data, filters);

    // Perform the specified aggregation
    var field = query["field"].GetValue<string>();
    var operation = query["operation"].GetValue<string>();
    var result = operation switch
    {
        "sum" => filteredData.Sum(item => item[field].GetValue<decimal>()),
        "average" => filteredData.Average(item => item[field].GetValue<decimal>()),
        "count" => filteredData.Count(),
        _ => throw new NotSupportedException($"Operation {operation} is not supported")
    };

    return result.ToString();
}

In [ ]:
var structuredQuery = await GetStructuredQueryAsync(userQuery);

Console.WriteLine($"Structured Query: {structuredQuery}");

var result = ExecuteStructuredQuery(jsonObject, structuredQuery);
Console.WriteLine($"Result: {result}");